In [10]:
%matplotlib inline

In [11]:
import pandas as pd
import bqplot
from bqplot import *
import us
import bqplot.pyplot as plt
from ipywidgets import widgets
import ipywidgets

In [12]:
names = ["date", "city", "state", "country", "shape", "duration_seconds",
         "duration_reported", "description", "report_date", "latitude",
         "longitude"]

fn = 'ufo-scrubbed-geocoded-time-standardized.csv'
ufo = pd.read_csv(fn, names = names, parse_dates = ["date", "report_date"])

abbr_to_fits = us.states.mapping('abbr', 'fips')
ufo["fips"] = ufo["state"].apply(lambda a: int(abbr_to_fits.get(str(a).upper(), -1)))
ufo_date = ufo.set_index("date")
ufo['year'] = ufo_date.index.year
ufo = ufo[ufo['fips'] != -1]

In [13]:
fips_count = ufo.groupby(['fips','year'])['duration_seconds'].count()
fips_sum = ufo.groupby(['fips','year'])['duration_seconds'].sum()
fips = pd.DataFrame()

In [14]:
fips['total_sightings'] = fips_count
fips['duration_seconds'] =fips_sum
fips['state'] = ufo['state']

In [15]:
fips = fips.reset_index(level=['fips','year'])

In [16]:
@ipywidgets.interact(field = ["total_sightings", "duration_seconds"], year = (min(fips['year']),max(fips['year']),1))
def make_plot(field, year=max(fips['year'])):
    fips_year = fips[fips['year']==int(year)]
    #fips_select = fips_year['fips'fips_year]
    map_fig = plt.figure("Visualizing ufo", min_width=1300, min_height=800)
    def_tt = Tooltip(fields=['name', 'color'], formats=['', ''], labels=['State', field])
    map_res = plt.geo(map_data=bqplot.topo_load('map_data/USStatesMap.json'),
                      colors = {'default_color':'Black'},
                      scales = {'projection': bqplot.AlbersUSA(),
                                'color': ColorScale(colors=['Red','Gray','DeepSkyBlue'])},
                      color=dict(zip(fips_year['fips'],fips_year[field])),
                      tooltip=def_tt) # 
    
    
    plt.show()

A Jupyter Widget